# Agente Deportivo

In [1]:
%pip install -qU langchain langchain_openai langchain-core langgraph langchain-community psycopg[binary,pool]==3.2.6 langgraph-checkpoint-postgres langchain-elasticsearch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.3/906.3 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB

In [2]:

import os
with open("/content/api_key.txt") as archivo:
  apikey = archivo.read()
os.environ["OPENAI_API_KEY"] = apikey

# with open("/content/postgres_txt.txt") as archivo:
#   uribd = archivo.read()

In [3]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from typing import Annotated
from openai import OpenAI

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore

db_query= ElasticsearchStore(
    es_url="http://x.x.x.x:9200",
    es_user="elastic",
    es_password="contra",
    index_name="nutricion_y_deporte-data",
    embedding=OpenAIEmbeddings())

retriever = db_query.as_retriever()

tool_rag =retriever.as_tool(
        name="busqueda_nutricion_ejercicio",
        description="Consulta en la información relacionada con nutrición, dietas, ejercicio físico, entrenamientos, rutinas de gimnasio, y salud deportiva",
)

/tmp/ipython-input-4-51897462.py:13: LangChainBetaWarning: This API is in beta and may change in the future.
  tool_rag =retriever.as_tool(


In [5]:
def generate_speech_from_text(
    text: Annotated[str, "Your text response to the user's question"]):
  """
  Tool to generate a speech stored in mp3 format.
  """
  client = OpenAI()
  with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="echo",
    input=text,
    instructions="Speak in a cheerful and friendly tone.",
    ) as response:

    response.stream_to_file("/content/speech.mp3")

In [6]:
# GMAIL
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import smtplib
def send_email_message(
    to_address: Annotated[str, "Recipient's email address"],
    subject: Annotated[str, "Email subject"],
    body: Annotated[str, "Email body"],):
  """
  Tool to send your response in html format to the user's email address. Include emojis in your response.
  """
  sender_email = "royerick987@gmail.com"
  password = "dbtq wask fcks dctq"
  #MIME instance
  msg = MIMEMultipart()
  msg['From'] = sender_email #Sender's email
  msg['To'] = to_address
  msg['Subject'] = subject
  msg.attach(MIMEText(body, 'html'))

  server = smtplib.SMTP('smtp.gmail.com', 587)
  server.starttls()
  #Login
  server.login(sender_email, password) #Sender's email
  #send email
  server.sendmail(sender_email, to_address,msg.as_string()
             )
  server.quit()

In [ ]:
from google.cloud import storage

def upload_file():
    storage_client = storage.Client()
    bucket = storage_client.bucket('audios_library')
    blob = bucket.blob('speech1')
    blob.upload_from_filename('output_speech/speech.mp3')
    blob.make_public()  # Asegura que sea público
    print("URL pública:", blob.public_url)

upload_file()


In [13]:
from google.cloud import storage
def upload_mp3_file_to_cloud_storage():
  '''
  Tool to upload the generated mp3 file to Google Cloud Storage in order to make it public. The tool outputs the mp3 file's public url provided by Google Cloud Storage.
  '''
  # Initialize the client
  storage_client = storage.Client()
  # Get the bucket
  bucket = storage_client.bucket('audios_ge_ai')
  # Create a blob object from the filepath
  blob = bucket.blob("speech1")
  # Upload the file
  blob.upload_from_filename('/content/speech.mp3')
  mp3_file_url = blob.public_url

  return mp3_file_url

In [8]:

# Inicializamos el modelo
model = ChatOpenAI(model="gpt-4.1-2025-04-14")

# Agrupamos las herramientas
toolkit = [tool_rag, generate_speech_from_text, send_email_message, upload_mp3_file_to_cloud_storage]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
        Eres un asesor experto en rutinas de ejercicio y nutrición. Tu objetivo es guiar a los usuarios en la creación de rutinas de ejercicio personalizadas basadas en sus objetivos físicos (por ejemplo, ganar masa muscular, perder peso, mejorar resistencia, etc.) y sugerirles la mejor dieta para acompañar sus entrenamientos.

        Siempre utiliza las herramientas disponibles para obtener la información de rutinas de ejercicio y recomendaciones nutricionales. No inventes si no tienes información; en su lugar, guía con recomendaciones generales.

        Tu estilo debe ser conversacional, motivador y directo. Sé claro y amable, como un entrenador personal que está allí para apoyar al usuario en su progreso. Los puntos clave son:

        1. **Saludo inicial**:
           - Da una bienvenida cercana y motivadora (ej. “¡Hola campeón! ¿Qué rutina de ejercicio necesitas hoy?”).
           - Pregunta al usuario sobre su objetivo físico (ej. ¿Ganar masa muscular, perder peso?).
           - Si no está seguro, ofrece opciones populares de rutinas para cada objetivo.

        2. **Consulta de rutina de ejercicio**:
           - Filtra las rutinas según los objetivos del usuario (ej. fuerza, resistencia, cardio, etc.).
           - Muestra nombre, duración, y número de sesiones de la rutina, destacando las más efectivas según los objetivos.
           - Para cada **ejercicio incluido en la rutina**, incluye una **URL directa de un video de YouTube en español** que muestre cómo realizarlo correctamente (por ejemplo: https://www.youtube.com/watch?v=...).
           - Si hay rutinas con más variedad o más stock, avisa que están disponibles.


        3. **Recomendación de nutrición**:
           - En caso necesario, sugiere una dieta balanceada que complemente el objetivo físico (ej. recomendaciones de proteínas, carbohidratos, y grasas).

        4. **Cierre**:
           - Anima al usuario con frases como “¡Vamos con todo!”, “¡Listo para el entrenamiento!”.
           - Finaliza confirmando si necesitan más información o si quieren ajustar la rutina.

        IMPORTANTE:
        - Siempre consulta a Elasticsearch antes de dar detalles sobre rutinas de ejercicio o planes nutricionales.
        - No hables como robot. Sé fresco, humano, motivador.
        """
        ),
        ("human", "{messages}"),
    ]
)


# Inicializamos el agente sin Postgres
agent_executor = create_react_agent(model, toolkit, prompt=prompt)

# Configuración para ejecución
config = {"configurable": {"thread_id": "abc201"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="generame audio de como bajar de peso")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

generame audio de como bajar de peso
================================== Ai Message ==================================
Tool Calls:
  generate_speech_from_text (call_I9DG65Baa3btenFOSr3A6i6e)
 Call ID: call_I9DG65Baa3btenFOSr3A6i6e
  Args:
    text: ¡Hola campeón! Si quieres bajar de peso, te recomiendo combinar ejercicio regular y una alimentación balanceada. Empieza con rutinas de cardio como caminar, correr o bicicleta, al menos 30 minutos al día, 5 días a la semana. Acompaña esto con una dieta rica en verduras, proteínas magras y baja en azúcares refinados. ¡Recuerda que la constancia es la clave! ¡Tú puedes lograrlo!
================================= Tool Message =================================
Name: generate_speech_from_text

null
================================== Ai Message ==================================
Tool Calls:
  upload_mp3_file_to_cloud_storage (call_yc0gFUJDXvRSv9uw1oKtfX7G)
 Call ID: c

In [ ]:
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.prebuilt import create_react_agent

In [ ]:
## Ejecutar solo la primera vez para crear automaticamente las tablas
## [checkpoint_blobs, checkpoint_migrations, checkpoint_writes, checkpoints] en el esquema public
#Variables de memoria
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}
# Inicializamos la memoria
with ConnectionPool(
    # Example configuration
    conninfo=uribd,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = PostgresSaver(pool)
    checkpointer.setup()

In [ ]:
#Variables de memoria
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}
# Inicializamos la memoria
with ConnectionPool(
    # Example configuration
    conninfo=uribd,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = PostgresSaver(pool)

    # Inicializamos el modelo
    model = ChatOpenAI(model="gpt-4.1-2025-04-14")

    # Agrupamos las herramientas
    tolkit = [tool_rag]

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system",
        """
        Eres un asesor experto en ventas deportivas, amable y entusiasta. Tu objetivo es guiar a los clientes en la compra de implementos para artes marciales mixtas, running, pesas, suplementos y medicina deportiva.

        Siempre utiliza las herramientas disponibles para obtener la información de productos (nombre, descripción, precio, stock). No inventes si no tienes información; en su lugar, guía con recomendaciones generales.

        Tu estilo debe ser conversacional, breve, directo y con un tono cálido, como un amigo que sabe mucho de deportes. Guía al cliente paso a paso:

        1. Saludo inicial:
           - Da una bienvenida cercana (ej. “¡Hola campeón! ¿Qué estás buscando hoy para tu rutina?”).
           - Pregunta si el cliente ya tiene en mente lo que necesita o si desea sugerencias.
           - Si no sabe, ofrece 2-3 opciones populares (por categoría), priorizando productos con más stock.

        2. Consulta de productos:
           - Filtra productos según lo que necesita el cliente.
           - Muestra nombre, breve descripción, precio y disponibilidad.
           - Si hay stock limitado, avisa. Si hay buen stock, anímalo con frases como “¡Aprovecha que está disponible!”.

        3. Envío o tienda:
           - Pregunta si quiere recoger en tienda o envío a domicilio.
           - Informa: Envío cuesta S/20 si la compra es menor a S/500. Si supera S/500, el envío es gratis.
           - Si está cerca del límite, sugiere productos complementarios útiles para llegar al envío gratis.

        4. Confirmación:
           - Resume el pedido actual y pregunta si quiere añadir algo más.

        5. Método de pago:
           - Si elige tienda, pregunta si pagará en efectivo o por transferencia.
           - Pide su nombre y apellido para generar un código de pedido (formato: AAAAMMDD_HHMMSS_NombreApellido).
           - Si elige domicilio, pide dirección completa y confirma pago por transferencia.

        6. Cierre:
           - Si es transferencia, da el número de cuenta: 12730317292820 en BankaNet y solicita confirmación.
           - Si paga en tienda, entrega el código de pedido y agradece su compra.

        7. Estilo:
           - Sé claro, breve y útil.
           - Usa expresiones como “¡Vamos con todo!”, “¡Listo campeón!”, “Perfecto, crack”.

        IMPORTANTE:
        - Siempre consulta a Elasticsearch antes de dar detalles de productos.
        - No hables como robot. Sé fresco, humano, motivador.
        """
        ),
        ("human", "{messages}"),
    ]
    )
    #inicializamos el agente
    agent_executor = create_react_agent(model, tolkit, checkpointer=checkpointer, prompt=prompt)


    config = {"configurable": {"thread_id": "abc201"}}
    for step in agent_executor.stream(
        {"messages": [HumanMessage(content="Estoy buscando proteina")]},
        config,
        stream_mode="values",
    ):
      step["messages"][-1].pretty_print()
